# 학교 500m 이내 경찰관서수 계산
- data 
    - 나이스_서울_전체_위경도_학교기본정보_Ver2.csv
    - 경찰청_2021_서울_경찰관서위치정보.csv
- data(output)
    - 학교500m_cctv_경찰서_개수_Ver2.csv

In [53]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
from sklearn.preprocessing import StandardScaler
from haversine import haversine
from tqdm import tqdm

In [54]:
# 나이스 학교 데이터 불러오기
school_loc_data = pd.read_csv("D:\MultiCampus\DATAexam\융복합\폭력분석\data\나이스_서울_전체_위경도_학교기본정보_Ver2.csv", index_col=0)
school_loc_data.head(1)

,시도교육청코드,시도교육청명,표준학교코드,학교명,영문학교명,학교종류명,소재지명,시군구,관할조직명,설립명,...,산업체특별학급존재여부,고등학교일반실업구분명,특수목적고등학교계열명,입시전후기구분명,주야구분명,설립일자,개교기념일,수정일,위도,경도
0,B10,서울특별시교육청,7010057,가락고등학교,Garak High School,고등학교,서울특별시,송파구,서울특별시교육청,공립,...,N,일반계,NaN,후기,주간,19881223,19890428,20230115,37.501076,127.116426


In [56]:
police_loc_data = pd.read_csv("D:\MultiCampus\DATAexam\융복합\폭력분석\data\경찰청_2021_서울_경찰관서위치정보.csv", index_col=0)
police_loc_data.head()

,지방청,경찰서,관서명,구분,주소,위도,경도
0,서울청,서울중부,을지,지구대,서울특별시 중구 을지로 234,37.566475,127.003942
1,서울청,서울중부,광희,지구대,서울특별시 중구 퇴계로 375-1,37.565268,127.013376
2,서울청,서울중부,약수,지구대,"서울특별시 중구 동호로 5길 15, 약수지구대",37.552144,127.012382
3,서울청,서울중부,신당,파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)",37.565044,127.016385
4,서울청,서울중부,장충,파출소,서울특별시 중구 동호로 261,37.558763,127.004802


In [57]:
# 경찰관서수 구하는 함수 
def close_police(school_loc_data, police_loc_data): 
    schoolloc, policeloc, police_cnt = [], [], []
    for col in tqdm(school_loc_data.itertuples()): 
        schoolloc = [col.위도, col.경도]
        distance = []
        for col2 in police_loc_data.itertuples(): 
            policeloc = [col2.위도, col2.경도]
            distance.append(haversine(schoolloc, policeloc, unit = 'm'))
            
        distance = [d for d in distance if d <= 500] 
        police_cnt.append(len(distance))
    return police_cnt

In [58]:
# school_loc_data = close_police(school_loc_data, police_loc_data, "500미터내_파출소개수")
school_loc_data['500m_inner_police_dept'] = close_police(school_loc_data, police_loc_data)
school_loc_data

320it [00:00, 725.92it/s]


,시도교육청코드,시도교육청명,표준학교코드,학교명,영문학교명,학교종류명,소재지명,시군구,관할조직명,설립명,...,고등학교일반실업구분명,특수목적고등학교계열명,입시전후기구분명,주야구분명,설립일자,개교기념일,수정일,위도,경도,500m_inner_police_dept
0,B10,서울특별시교육청,7010057,가락고등학교,Garak High School,고등학교,서울특별시,송파구,서울특별시교육청,공립,...,일반계,NaN,후기,주간,19881223,19890428,20230115,37.501076,127.116426,0
1,B10,서울특별시교육청,7011169,가재울고등학교,Gajaeul High School,고등학교,서울특별시,서대문구,서울특별시교육청,공립,...,일반계,NaN,후기,주간,20130301,20130501,20230115,37.574364,126.909530,0
2,B10,서울특별시교육청,7010117,강동고등학교,Kangdong High School,고등학교,서울특별시,강동구,서울특별시교육청,사립,...,일반계,NaN,후기,주간,19841217,19850908,20230115,37.549917,127.160668,0
3,B10,서울특별시교육청,7010118,강서고등학교,Gangseo High School,고등학교,서울특별시,양천구,서울특별시교육청,사립,...,일반계,NaN,후기,주간,19831101,19831101,20230115,37.536316,126.865771,1
4,B10,서울특별시교육청,7010270,강서공업고등학교,Gangseo Technical High School,고등학교,서울특별시,강서구,서울특별시교육청,공립,...,전문계,NaN,전기,주간,19940105,19940105,20230115,37.576034,126.816182,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,B10,서울특별시교육청,7010766,효문고등학교,Hyomun High School,고등학교,서울특별시,도봉구,서울특별시교육청,공립,...,일반계,NaN,후기,주간,20051229,20060605,20230115,37.652699,127.018400,0
316,B10,서울특별시교육청,7010292,휘경공업고등학교,Hwigyeong Technical High School,고등학교,서울특별시,동대문구,서울특별시교육청,공립,...,전문계,NaN,전기,주간,19971220,19980702,20230115,37.585153,127.069451,1
317,B10,서울특별시교육청,7010267,휘경여자고등학교,Hwikyung Girls` High School,고등학교,서울특별시,동대문구,서울특별시교육청,사립,...,일반계,NaN,후기,주간,19750303,19750519,20230115,37.581901,127.067434,1
318,B10,서울특별시교육청,7010268,휘문고등학교,Whimoon High School,고등학교,서울특별시,강남구,서울특별시교육청,사립,...,일반계,NaN,전기,주간,19060501,19060501,20230115,37.505289,127.061994,0


In [59]:
school_loc_data['500m_inner_police_dept'].value_counts()

0    213
1     88
2     19
Name: 500m_inner_police_dept, dtype: int64

In [60]:
school_loc_data.to_csv("D:\MultiCampus\DATAexam\융복합\폭력분석\data\학교500m_cctv_경찰서_개수_Ver2.csv", index=False, encoding='utf-8-sig')

In [41]:
# tmp = pd.read_csv("D:\MultiCampus\DATAexam\융복합\폭력분석\data\학교500m_cctv_경찰서_개수.csv")
# tmp[['500미터내_파출소개수']].value_counts()

500미터내_파출소개수
0               191
1               108
2                21
dtype: int64